### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from tqdm import tqdm
from pandas.arrays import IntervalArray

import mne
from mne.time_frequency import tfr_array_morlet
from scipy.stats import zscore

from utils__helpers_macro import hilbert_powerphase, hilbert_envelope

In [2]:
import utils__config
os.chdir(utils__config.working_directory)
os.getcwd()

'G:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep\\Revisions'

### Parameters
Please note that .FIF files that are split (due to the 2GB size limit) will have their original name saved internally as well, so manually renaming files at a later date will result in being unable to load the file, as the first file in the split will have its original internal name that serves as a template for finding split files.

In [3]:
# fif_path = 'Data/S01_Feb02_256hz.fif'
# spike_path = 'Data/S01_Feb02_spike_times.csv'
# hypno_path = 'Data/S01_Feb02_hypnogram.csv' 
# legui_path = 'Data/S01_electrodes.csv'
# bad_channel_path = 'Data/S01_Feb02_bad_channels.csv'
# output_path = 'Cache/S01_Feb02_sf_coupling.csv'

fif_path = 'Data/S05_Jul11_256hz.fif'
spike_path = 'Data/S05_Jul11_spike_times.csv'
hypno_path = 'Data/S05_Jul11_hypnogram.csv' 
legui_path = 'Data/S05_electrodes.csv'
bad_channel_path = 'Data/S05_Jul11_bad_channels.csv'
output_path = 'Cache/S05_Jul11_sf_coupling.csv'

# fif_path = 'Data/S05_Jul12_256hz.fif'
# spike_path = 'Data/S05_Jul12_spike_times.csv'
# hypno_path = 'Data/S05_Jul12_hypnogram.csv' 
# legui_path = 'Data/S05_electrodes.csv'
# bad_channel_path = 'Data/S05_Jul12_bad_channels.csv'
# output_path = 'Cache/S05_Jul12_sf_coupling.csv'

# fif_path = 'Data/S05_Jul13_256hz.fif'
# spike_path = 'Data/S05_Jul13_spike_times.csv'
# hypno_path = 'Data/S05_Jul13_hypnogram.csv' 
# legui_path = 'Data/S05_electrodes.csv'
# bad_channel_path = 'Data/S05_Jul13_bad_channels.csv'
# output_path = 'Cache/S05_Jul13_sf_coupling.csv'

In [4]:
sampling_freq = 256

### Load Data

In [5]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove rejected channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))

# Load LeGUI data
legui = pd.read_csv(legui_path)
legui = legui[['elec_label', 'hemisphere', 'roi_1']]
legui.columns = ['channel', 'laterality', 'region']

# Load Spike data
spikes = pd.read_csv(spike_path)
spikes = spikes[['unit_id', 'seconds', 'unit_laterality', 'unit_region']]

Opening raw data file Data/S05_Jul11_256hz.fif...


C:\Users\layto\AppData\Local\Temp\ipykernel_17320\1663171324.py:1: RuntimeWarning: This filename (Data/S05_Jul11_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


    Range : 0 ... 7249663 =      0.000 ... 28318.996 secs
Ready.
Opening raw data file G:\My Drive\Residency\Research\Lab - Damisah\Project - Sleep\Revisions\Data\S05_Jul11_256hz-1.fif...
    Range : 7249664 ... 8921599 =  28319.000 ... 34849.996 secs
Ready.
Reading 0 ... 8921599  =      0.000 ... 34849.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


### Extract slow-wave-band phase and delta power

In [6]:
# Extract Slow-Wave-band Phase
swb = raw.copy()
swb = hilbert_powerphase(data = swb, lower = 0.3, upper = 1.5, njobs = 6)
swb = swb[['time', 'channel', 'power', 'phase']]
swb.columns = ['time', 'channel', 'swb_power', 'swb_phase']

# Extract Delta Power
delta = raw.copy()
delta = hilbert_powerphase(data = delta, lower = 0.3, upper = 4, njobs = 6)
delta = delta[['time', 'channel', 'power', 'phase']]
delta.columns = ['time', 'channel', 'd_power', 'd_phase']

# Merge slow-wave-band phase and delta power
lfp = swb.merge(delta, on = ['time', 'channel'])
lfp = lfp[['time', 'channel', 'swb_phase', 'd_power']]
lfp.columns = ['time', 'channel', 'phase', 'power']

# Log-normalize and z-score the delta power
lfp['log_power'] = 10 * np.log10(lfp['power'])
lfp['zlog_power'] = lfp.groupby(['channel'])['log_power'].transform(zscore)

# Merge with LeGUI data to get LFP laterality and region
lfp = lfp.merge(legui, on = 'channel', how = 'inner')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 1.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 1.50 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.50 Hz)
- Filter length: 2817 samples (11.004 s)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    3.0s
[Parallel(n_jobs=6)]: Done  53 out of  53 | elapsed:   11.9s finished


Converting "channel" to "category"...
Converting "ch_type" to "category"...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 2817 samples (11.004 s)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    1.7s
[Parallel(n_jobs=6)]: Done  53 out of  53 | elapsed:    6.6s finished


Converting "channel" to "category"...
Converting "ch_type" to "category"...


### Add sleep stage to LFP data

In [7]:
# Load hypnogram
hypno = pd.read_csv(hypno_path, header = None)
hypno = hypno.reset_index()
hypno.columns = ['sample', 'stage']
hypno['time'] = hypno['sample'] / sampling_freq

# Hypnogram dictionary: 
# (-2) = Unassigned
# (-1) = Artifact
# (0) = Awake
# (1) = N1
# (2) = N2
# (3) = N3
# (4) = REM

# Extract and group sleep stages
hypno['stage'] = np.where(hypno['stage'].isin([2, 3]), 'NREM', 'WREM')

# Merge with hypnogram
lfp = pd.merge_asof(lfp.sort_values('time'), hypno.sort_values('time'),
                    left_on = 'time', right_on = 'time', direction = 'nearest')

### Define DREM sleep (high delta power NREM)

In [8]:
# Calculate percentile of power (only NREM rows)
lfp['nrem_tile'] = np.nan
lfp.loc[lfp['stage'] == 'NREM', 'nrem_tile'] = lfp.loc[lfp['stage'] == 'NREM', 'power'].rank(pct=True)

# Generate DREM column based on 75th percentile of NREM power
lfp['DREM'] = np.where((lfp['nrem_tile'] >= 0.75) & (lfp['stage'] == 'NREM'), 1, 0)

### Intersect spikes with nearest values of LFP/hypno data

In [9]:
data = pd.DataFrame()

for chan in tqdm(lfp.channel.unique()):

    # Subset the phase-envelope dataset
    lfp_channel = lfp[lfp.channel == chan].copy(deep = True)

    # For every spike, find the nearest 
    # sample in the phase-envelope dataset...
    data_temp = pd.merge_asof(spikes.sort_values('seconds'), lfp_channel.sort_values('time'), 
                              left_on = 'seconds', right_on = 'time', direction = 'nearest')

    data_temp.drop(columns = ['time', 'sample'], inplace = True)

    # Concatenate into final dataset
    data = pd.concat((data, data_temp))

data = data[['stage', 'DREM', 'channel', 'laterality', 'region', 
             'unit_id', 'unit_laterality', 'unit_region', 
             'phase', 'power', 'seconds']]

100%|██████████| 53/53 [22:02<00:00, 24.96s/it]


In [10]:
# Further reduce columns to save space
data = data[['stage', 'DREM', 'channel', 'unit_id', 'phase', 'power']]

# Export
data.to_csv(output_path, index = False)

In [11]:
data

,stage,DREM,channel,unit_id,phase,power
0,WREM,0,LOF1,S05_Ch240_neg_Unit1,1.818339,0.003713
1,WREM,0,LOF1,S05_Ch234_neg_Unit2,1.818339,0.003713
2,WREM,0,LOF1,S05_Ch240_neg_Unit1,1.923831,0.004137
3,WREM,0,LOF1,S05_Ch240_neg_Unit1,1.990766,0.004375
4,WREM,0,LOF1,S05_Ch234_neg_Unit2,2.118140,0.004765
...,...,...,...,...,...,...
3862203,WREM,0,LAC7,S05_Ch194_neg_Unit4,-0.154210,0.000942
3862204,WREM,0,LAC7,S05_Ch210_neg_Unit2,-0.154210,0.000942
3862205,WREM,0,LAC7,S05_Ch194_neg_Unit4,0.287288,0.001512
3862206,WREM,0,LAC7,S05_Ch194_neg_Unit2,0.357987,0.001573
